In [ ]:
import time
import scipy.special
import numpy as np
import matplotlib.pyplot as plt
from pyscf import gto, scf, mp, fci, ci, cc

# set the method to use - should be one of HF, MP2, FCI, CISD, CCSD
method = 'HF'

# set the basis
basis = '6-31G'

# a convenient way to compress and expand the molecule from it's equilibrium length
# the first number is the minimal, the second the maximal and the third the step size
bond_scale = [0.96, 1.10, 0.02]

# define molecule parameters: H2
bond_length = 0.702
geometry = 'H 0 0 {}; H 0 0 {}'
spin = 0

# define molecule parameters: HF
# bond_length = 0.91
# geometry = 'H 0 0 {}; F 0 0 {}'
# spin = 0

# define molecule parameters: O2
# bond_length = 1.21
# geometry = 'O 0 0 {}; O 0 0 {}'
# spin = 2

# define molecule parameters: my favorite molecule
# geometry = '''
# C          1.13569        2.22607        0.00008
# C          2.34660        1.48225       -0.00315
# O          1.11076        3.45322        0.00297
# N          0.00000        1.41087        0.00000
# N          3.65481        1.87305       -0.00480
# C          2.38646        0.12104       -0.00500
# N          1.24110       -0.63754       -0.00365
# N          3.66420       -0.35109       -0.00778
# C          0.00000        0.00000        0.00000
# O         -1.05949       -0.63408        0.00333
# C         -1.30072        2.05116        0.00361
# C          1.32846       -2.08635       -0.00430
# H         -1.84850        1.73732        0.89829
# H         -1.86170        1.72312       -0.87769
# H         -1.23294        3.14176       -0.00582
# H          1.87131       -2.41138       -0.89754
# H          0.34073       -2.55462       -0.00693
# H          1.86704       -2.41210        0.89126
# C          4.41443        0.73303       -0.00765
# C          4.15051        3.22594       -0.00368
# H          3.78631        3.73087       -0.90182
# H          5.24349        3.21375       -0.00572
# H          3.78964        3.72816        0.89731
# H          5.49723        0.74732       -0.00947
# '''
# spin = 0

# define your own molecules here if you want

# loop over bond scales
energies = []
bond_lengths = []
for scale in np.arange(*bond_scale):

    # start a timer
    start_time = time.time()

    # build the molecule and set the verbosity level
    mol = gto.M(atom    = geometry.format(-0.5 * bond_length * scale, 0.5 * bond_length * scale),
                basis   = basis,
                spin    = spin,
                verbose = 0)

    # output some information on the calculation
    print("{:32s}: {:.3f}".format('Bond scale',         scale))
    print("{:32s}: {:.3f} Å".format('Bond length',      bond_length * scale))
    print("{:32s}: {:}".format('Number of atoms',       mol.natm))
    print("{:32s}: {:}".format('Number of electrons',   mol.nelectron))
    print("{:32s}: {:}".format('Number of GTOs',        mol.nbas))
    print("{:32s}: {:}".format('Number of GFs',         sum([i[2] for i in mol._bas])))
    print('{:32s}: {:d}'.format('Spin orbitals',        mol.nbas * 2))
    print('{:32s}: {:d}'.format('Electrons',            mol.nelectron))
    print('{:32s}: {:d}'.format('Total determinants',   scipy.special.comb(mol.nbas * 2, mol.nelectron, exact=True)))
    for i in range(mol.nelectron+1):
        count = scipy.special.comb(mol.nelectron, i, exact=True) * scipy.special.comb(mol.nbas * 2 - mol.nelectron, i, exact=True)
        print('{:-5d}-fold excited determinants : {:d}'.format(i, count))

    # run the specified calculations
    if method == 'HF':
        hf_calc = scf.UHF(mol)
        res = hf_calc.scf()
        print('{:32s}: {:.4f} Ha'.format('HF energy', res))

    elif method == 'MP2':
        mp2_calc = mp.MP2(scf.UHF(mol)).as_scanner()
        res = mp2_calc(mol)
        print('{:32s}: {:.4f} Ha'.format('MP2 energy', res))

    elif method == 'FCI':
        hf_calc = scf.RHF(mol) # we cheat a bit here because the pyscf CI solver only accepts RHF
        hf_calc.kernel()
        ci_calc = fci.FCI(mol, hf_calc.mo_coeff)
        res = ci_calc.kernel()
        res = res[0] # extract the ground state energy
        print('{:32s}: {:.4f} Ha'.format('Full CI energy', res))

    elif method == 'CISD':
        ci_calc = ci.CISD(scf.UHF(mol)).as_scanner()
        res = ci_calc(mol)
        print('{:32s}: {:.4f} Ha'.format('CISD energy', res))

    elif method == 'CCSD':
        cc_calc = cc.CCSD(scf.UHF(mol)).as_scanner()
        res = cc_calc(mol)
        print('{:32s}: {:.4f} Ha'.format('CCSD energy', res))

    else:
        print('*** Error: Unknown method! Did you set it correctly? ***')

    bond_lengths.append(scale * bond_length)
    energies.append(res)
        
    # measure and output elapsed time 
    elapsed_time = time.time() - start_time
    print('{:32s}: {:.2f} sec'.format('Elapsed time', elapsed_time))
    print('-------------------------------------------------------')

#print and plot the results
print()
print('---------------+-----------------+------------')
print('Bond scale (-) | Bond length (Å) | Energy (Ha)')
print('---------------+-----------------+------------')
for i in range(len(bond_lengths)):
    print('{:14.3f} | {:15.3f} | {:11.4f}'.format(bond_lengths[i] / bond_length, bond_lengths[i], energies[i]))
fig, ax = plt.subplots()
ax.plot(bond_lengths, energies, marker = 'o')
ax.set_title(method)
ax.set_xlabel('Bond length (Å)')
ax.set_ylabel('Energy (Ha)')
plt.show()